In [ ]:
import csv
import numpy as np
import pandas as pd

In [ ]:
x = []
y = []
with open ('/content/data-overweight.csv') as file:
  csv_reader = csv.reader(file)
  for row in csv_reader:
    if row[0]=='ID':
      continue
    datum = []
    datum.append(float(row[3]))
    datum.append(float(row[4]))
    if row[1]=='Female':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[2]))
    bmi = float(row[4])/(float(row[3])**2)
    datum.append(bmi)
    if row[5]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    if row[6]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[7]))
    datum.append(float(row[8]))
    if row[9]=='no':
      datum.append(0)
    elif row[9]=='Sometimes':
      datum.append(1)
    elif row[9]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    #if row[10]=='yes':
      #datum.append(0)
    #else:
      #datum.append(1)
    datum.append(float(row[11]))
    if row[12]=='yes':
      datum.append(0)
    else:
      datum.append(1)
    datum.append(float(row[13]))
    datum.append(float(row[14]))
    if row[15]=='no':
      datum.append(0)
    elif row[15]=='Sometimes':
      datum.append(1)
    elif row[15]=='Frequently':
      datum.append(2)
    else:
      datum.append(3)
    if row[16]=='Walking':
      datum.append(0)
    elif row[16]=='Bike':
      datum.append(1)
    elif row[16]=='Motorbike':
      datum.append(2)
    elif row[16]=='Automobile':
      datum.append(3)
    else:
      datum.append(4)
    if row[17]=='Insufficient_Weight':
      y.append(0)
    elif row[17]=='Normal_Weight':
      y.append(1)
    elif row[17]=='Overweight':
      y.append(2)
    elif row[17]=='Obesity_Type_I':
      y.append(3)
    elif row[17]=='Obesity_Type_II':
      y.append(4)
    else:
      y.append(5)
    x.append(datum)
print(x)
print(y)
x = np.array(x)
y = np.array(y)

In [ ]:
!pip install optuna

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import optuna
from optuna.samplers import TPESampler
import pickle

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
base_model = lgb.LGBMClassifier()
base_model.fit(X_train, y_train)

In [ ]:
y_pred = base_model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 6,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

In [ ]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=500)

In [ ]:
print('Best parameters:', study.best_params)

In [ ]:
print('Best value:', study.best_value)

In [ ]:
print('Best trial:', study.best_trial)

In [ ]:
model = lgb.LGBMClassifier(**study.best_params)
model.fit(X_train, y_train,  feature_name=['Height', 'Weight', 'Gender', 'Age', 'BMI', 'family_history', 'FCHCF', 'FCV',
                                           'NMM','CFBM', 'CW', 'CCM', 'PAF', 'TUT', 'CA', 'Transportation'])

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
!pip install shap

In [ ]:
import shap
import matplotlib.pyplot as plt
fig = plt.figure()
shap_values = shap.TreeExplainer(model).shap_values(X_test)
shap.summary_plot(shap_values, X_test, feature_names = ['Height', 'Weight', 'Gender', 'Age', 'BMI', 'family_history', 'FCHCF', 'FCV',
                                                        'NMM', 'CFBM', 'Smoke', 'CW', 'CCM', 'PAF', 'TUT', 'CA', 'Transportation']
                                     , class_names = ['Insufficient_Weight', 'Normal_Weight', 'Overweight',
                                                         'Obesity_Level_I', 'Obesity_Level_II', 'Obesity_Level_III'], show = False
                                     , class_inds = model.classes_)
plt.gcf().set_size_inches(20,10)
plt.show()